# 创建训练镜像

## 创建项目目录

我们先创建一个空文件夹，作为项目根目录。

```shell
mkdir -p /tmp/aip-images/create-training-image
```

## 创建项目配置

编写项目配置 `pyproject.toml` 如下：
```toml
[tool.poetry]
name = "create-training-image"
version = "0.1.0"
description = ""
authors = ["ChouUn <chouunsoft@gmail.com>"]

[tool.poetry.dependencies]
python = "3.9.*"
tensorflow = "^2.9.1"
numpy = "^1.23.1"
boto3 = "^1.24.31"

[build-system]
# Should be the same as `$POETRY_VERSION`:
requires = ["poetry>=1.0"]
build-backend = "poetry.masonry.api"
```

## 创建库依赖锁

并根据配置文件 `pyproject.toml` 生成库依赖锁 `poetry.lock`。
生成命令如下：
```shell
poetry lock --no-update
```

## 创建构建配置

编写构建配置 `Dockerfile` 如下：
```dockerfile
FROM python:3.9

ARG ENV

ENV ENV=${ENV} \
  PYTHONFAULTHANDLER=1 \
  PYTHONUNBUFFERED=1 \
  PYTHONHASHSEED=random \
  PIP_NO_CACHE_DIR=off \
  PIP_DISABLE_PIP_VERSION_CHECK=on \
  PIP_DEFAULT_TIMEOUT=100 \
  POETRY_VERSION=1.1.14

# System deps:
RUN pip install "poetry==$POETRY_VERSION"

# Copy only requirements to cache them in docker layer
WORKDIR /opt/create-training-image/
COPY poetry.lock pyproject.toml ./

# Project initialization:
RUN poetry config virtualenvs.create false \
  && poetry install $(test "$ENV" == production && echo "--no-dev") --no-interaction --no-ansi

# Creating folders, and files for a project:
# COPY . /app

ENTRYPOINT ["/usr/local/bin/_entrypoint.sh"]
```
这里要注意，基本镜像不能选择 `Alpine Linux`，因为他使用了 `musl libc` 而不是 `glibc`，会导致 `numpy` 等依赖无法正常安装。

## 推送镜像

推送镜像到镜像仓库 `Harbor` 上。
此处 `aip-images` 是 `AIP` 的项目，用户可以选择路径。

```shell
docker build --tag harbor.nadileaf.com/aip-images/create-training-image:0.1.1 .
docker push harbor.nadileaf.com/aip-images/create-training-image:0.1.1
```